In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd

In [0]:
downloaded = drive.CreateFile({'id':'1UDiOho3fJT_VRssVjn9ZI9SrBI34Fkh9'}) ## Load Secondary_Source.csv in the edge_list_creation folder
downloaded.GetContentFile('secondary_sources.csv')  
file_dictionary = pd.read_csv('secondary_sources.csv')
del file_dictionary['Unnamed: 0']

In [0]:
file = file_dictionary.values   ## this cell is for putting all the files in a dictionary shape.
file_dictionary = {}
for i in file:
  file_dictionary[i[0]] = i[1]

In [0]:
file_dictionary['vandijk1976.txt'] ## Now we can check the key to be the file name, the content is matched to the key

'ee\n-— nance etn\naaaeneet ae\ntneae\nginning ARATE, OEcee\nd’Esnunna. et d’Elam l\'histoire sur Remarques\n(Rome). Dijk van J. Par\nMalgi?um. Elam Sukkalmah?! le Kuduzulus, 1.\nimprimés sceaux cylindres, des légendes Les\npar publiés judiciaires protocoles deux les sur\nin Teats des 34 et 33 numé:os les comme moi\nlisent: seIV, vol. Museum, Iraq the\n33: IV TIM\n-x! nu-uir- 1.\nu-4dsin-se-me-e dum\n]u-z[u-]Ju-u[8] [ir]-k[u-d\nja-ab-hu-rum 2.\n]-e-dadad [he dumu-ié-\nir-ku-du-zu-lu-us\nIha-ja-r[a-b]¢ 3.\nx] dumu-ja-hi-i8-[x\nir-4[\n[r]e!-2-di8-ha-ra 4.\nSINANNA.MUL §4 ir\nzi-im-ra-ha-mu-% 5.\n’dumu-la-PI-ih-el\nle Utumansum, 8, 34, No du lexception\ndu ceux correspondent noms ces réclamant,\njuges: des collége\n3; ha-am-du-u le nu-ri-ja, ss.: 18 33, IV TIM\nlzi-im- 1un-nu-bu; GIR.NITA; le \\ja-ab-hu-rum,\nha-jag-ra-bt. Ya-ku-nu; ra-am-mu;\nha-am-du-u, le nu-ri-ja, ss.: 34, IV TIM\ndadad-mu-ba-. 1! GIR.NITA; le \\ja-ab-hu-rum,\nYa-ku-nu; ?-4ISTAR; ISIPA ‘un-nu-bu-um li-it;\n—Tha-jag-ra

get help from: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
### Get the stopwords from different languages from nltk package
french_stopwords = stopwords.words('french')
german_stopwords = stopwords.words('german')
turkish_stopwords = stopwords.words('turkish')

In [0]:
## This is a function designed to remove all the stopwords in our text
def preprocess(text): # one little problem, gensim.utils.simple_preprocess gets rid off "I" automatically. 
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
          if token not in french_stopwords:
            if token not in german_stopwords:
              if token not in turkish_stopwords:
                result.append(token)
    return result

In [0]:
for key in file_dictionary:
  print("working on file "+ key)
  file_dictionary[key] = preprocess(file_dictionary[key])

working on file Llop_2010_AuOr_28_Rec._OBO_160_5.txt
working on file pardee2001b.txt
working on file Hecker_1978b_OrNS47_tib'imma.txt
working on file pardee1979-80a.txt
working on file Lassen, Agnete - Glyptic Encounters Appendicies.txt
working on file Yilmaz_2007_AMMY2006.txt
working on file Anastasio, Stefano - The Archaeology of Upper Mesopotamia. Subartu 1, 1995 (grey).txt
working on file Dercksen_1997_ArAn3_BilgicFS_ikribu.txt
working on file Günbatti_2008_AOF 35-1_An Eponym List - KEL G - from Kültepe.txt
working on file Goetze_1937_JAOS57_Subartu.txt
working on file kuhrt1988.txt
working on file Günbatti, Cahit - Die Beziehungen zwischen assyrischen Kaufleuten und anatolischen Fürsten aus Kültepe. CDOG 8, 2008 125-136.txt
working on file p249_s_a.txt
working on file Garelli_Michel_1994_AMMY93_tablets_Ankara.txt
working on file Goetze_1953b_Language29_PN.txt
working on file Kienast_1975_WdO8_Pfandrecht.txt
working on file PRU 3 plates.txt
working on file OzgucN_1994_HroudaFS

In [0]:
test_file = file_dictionary['vandijk1976.txt'] # an example to show what the file is stored in the dictionrary after preprocessed.

In [0]:
processed_files = []
for key in file_dictionary:
  processed_files.append(file_dictionary[key])

In [0]:
dictionary = gensim.corpora.Dictionary(processed_files)

In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_files]

In [0]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [0]:
topic_number = 10 # how many topic you want, by default 10 topics

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_number, id2word=dictionary, passes=2, workers=2)

In [0]:
for idx, topic in lda_model.show_topics(num_topics=10, num_words=5, log=False, formatted=True): ## change num_words to see number of words you want to see in a topic
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.003*"barley" + 0.003*"nome" + 0.003*"aššur" + 0.002*"hittite" + 0.002*"egypt"
Topic: 2 
Words: 0.004*"ofthe" + 0.004*"silver" + 0.003*"seal" + 0.003*"letter" + 0.002*"year"
Topic: 3 
Words: 0.004*"receipt" + 0.004*"account" + 0.003*"ugarit" + 0.003*"dumu" + 0.003*"plus"
Topic: 4 
Words: 0.006*"silber" + 0.005*"fiir" + 0.003*"babbar" + 0.003*"siegel" + 0.002*"oben"
Topic: 5 
Words: 0.004*"argent" + 0.003*"aššur" + 0.003*"hittite" + 0.003*"ofthe" + 0.003*"silver"
Topic: 6 
Words: 0.003*"ware" + 0.003*"area" + 0.003*"level" + 0.003*"site" + 0.003*"wall"
Topic: 7 
Words: 0.003*"tempel" + 0.003*"liste" + 0.003*"hittite" + 0.002*"texte" + 0.002*"tell"
Topic: 8 
Words: 0.003*"level" + 0.003*"anatolian" + 0.003*"anatolia" + 0.002*"area" + 0.002*"wall"
Topic: 9 
Words: 0.007*"seal" + 0.004*"level" + 0.003*"style" + 0.002*"area" + 0.002*"anatolian"
Topic: 10 
Words: 0.003*"silver" + 0.002*"hittite" + 0.002*"level" + 0.002*"land" + 0.002*"type"


In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.003*"barley" + 0.003*"nome" + 0.003*"aššur" + 0.002*"hittite" + 0.002*"egypt" + 0.002*"egyptian" + 0.002*"temple" + 0.002*"adad" + 0.002*"royal" + 0.002*"come"
Topic: 2 
Words: 0.004*"ofthe" + 0.004*"silver" + 0.003*"seal" + 0.003*"letter" + 0.002*"year" + 0.002*"line" + 0.002*"veenhof" + 0.002*"copper" + 0.002*"trade" + 0.002*"babylonian"
Topic: 3 
Words: 0.004*"receipt" + 0.004*"account" + 0.003*"ugarit" + 0.003*"dumu" + 0.003*"plus" + 0.003*"fils" + 0.003*"textes" + 0.002*"cette" + 0.002*"deux" + 0.002*"comme"
Topic: 4 
Words: 0.006*"silber" + 0.005*"fiir" + 0.003*"babbar" + 0.003*"siegel" + 0.002*"oben" + 0.002*"dumu" + 0.002*"minen" + 0.002*"sohn" + 0.002*"zwei" + 0.002*"zeit"
Topic: 5 
Words: 0.004*"argent" + 0.003*"aššur" + 0.003*"hittite" + 0.003*"ofthe" + 0.003*"silver" + 0.002*"babbar" + 0.002*"dumu" + 0.002*"istar" + 0.002*"fils" + 0.002*"puzur"
Topic: 6 
Words: 0.003*"ware" + 0.003*"area" + 0.003*"level" + 0.003*"site" + 0.003*"wall" + 0.003*"north" + 0.0

In [0]:
top = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[],8:[],9:[], 10:[]}

In [0]:
for key in file_dictionary:
  topics = sorted(lda_model[bow_corpus[processed_files.index(file_dictionary[key])]], key = lambda tup: -1*tup[1])
  for i in topics:
    topic_number = i[0]+1
    inside = [key, i[1]]
    if i[1] > 0.5: ## change the threshold for topic weights
      top[topic_number].append(inside)

Then we make a dictionary (top) for the topics (topics) have been generated for each document (key) in the document set (fiile_dictionary)

In [0]:
Source = []
Target = []
Source_weight = []
Target_weight = []
Topic = []

In [0]:
for i in range(1, 11):
  for j in range(len(top[i])):
    for k in range(j+1, len(top[i])):
      if top[i][j][1] >= top[i][k][1]:
        Source.append(top[i][j][0])
        Target.append(top[i][k][0])
        Source_weight.append(top[i][j][1])
        Target_weight.append(top[i][k][1])
      else:
        Source.append(top[i][k][0])
        Target.append(top[i][j][0])
        Source_weight.append(top[i][k][1])
        Target_weight.append(top[i][j][1])
      Topic.append(i)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Topic': Topic})

In [0]:
Type = ['directed'] * len(result)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Type': Type, 'Topic': Topic})

In [0]:
result 

,Source,Target,Source_Weight,Target_Weight,Type,Topic
0,Pomponio_1997_SEL14_quisling_babilonesi.txt,"Carruba, Giorgieri & Mora eds - Atti del il co...",0.999690,0.722742,directed,1
1,Liverani_1988_Antico_Storica_OA_Chapters.txt,"Carruba, Giorgieri & Mora eds - Atti del il co...",0.999770,0.722742,directed,1
2,Morrisondiss.txt,"Carruba, Giorgieri & Mora eds - Atti del il co...",0.984790,0.722742,directed,1
3,Zamora_2000_SEL17_Misura_Shir_Shd_admin_txts_U...,"Carruba, Giorgieri & Mora eds - Atti del il co...",0.999664,0.722742,directed,1
4,liverani1998a.txt,"Carruba, Giorgieri & Mora eds - Atti del il co...",0.999653,0.722742,directed,1
...,...,...,...,...,...,...
227290,wilkinson2000.txt,"Wilkinson, T. J. - Archaeological Landscapes o...",0.963825,0.782746,directed,9
227291,"Stein, Gil - Hetrogeneity.txt",scnieder1977.txt,0.999875,0.765227,directed,9
227292,wilkinson2000.txt,scnieder1977.txt,0.963825,0.765227,directed,9
227293,"Stein, Gil - Hetrogeneity.txt",wilkinson2000.txt,0.999875,0.963825,directed,9


How many rows can a Google Sheet have?
Originally Answered: How many rows can you have in a google spreadsheet before performance becomes a serious problem? Short answer: 400,000 cells - http://docs.google.com/support/b... - Note the distinction between rows and cells. This means that you could have 40,000 rows of 10 columns or 80,000 rows of 5 columns.Oct 12, 2010

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
result.to_csv('Topic_edge_list2.csv', index=False)
!cp Topic_edge_list2.csv drive/My\ Drive/AWCA/Collab_notebooks/edge_list_creation/